In [5]:
import pandas as pd
import numpy as np

# =====================================================
# Track B - 금리 보정된 μ(mu) 추정 코드
#
# 목적:
# - 매매 실거래 데이터를 이용해
#   주택가격의 평균 성장률(μ)과 변동성(σ)을 추정
# - 이후 금리 변화에 대한 민감도(β)를 추정하여
#   현재 금리 환경을 반영한 보정 μ(mu_adj)를 계산
#
# 사용 데이터:
# - MD1(최종).csv : 매매 실거래 데이터
# - rate.csv      : 주택담보대출 기준금리(COFIX 등)
#
# Track B에서는 "가격 예측"이 목적이 아니라
# PD / LGD / EL 계산에 사용할 확률과정 파라미터를
# 현실적인 현재 시장 환경에 맞게 보정하는 것이 목적임
# =====================================================


# ===============================
# 0. 경로 설정
# ===============================
SALE_PATH = "/content/MD1(최종).csv"
RATE_PATH = "/content/rate.csv"

In [6]:
# ===============================
# 1. 매매 데이터 로드 및 전처리
# ===============================
sale = pd.read_csv(SALE_PATH)

# MD1 데이터 기준 컬럼 확정
DATE_COL  = "계약일"
PRICE_COL = "거래금액(만원)"
AREA_COL  = "전용면적(㎡)"

# 타입 변환
sale[DATE_COL]  = pd.to_datetime(sale[DATE_COL], errors="coerce")
sale[PRICE_COL] = pd.to_numeric(sale[PRICE_COL], errors="coerce")
sale[AREA_COL]  = pd.to_numeric(sale[AREA_COL], errors="coerce")

# 결측치 및 비정상 값 제거
sale = sale.dropna(subset=[DATE_COL, PRICE_COL, AREA_COL])
sale = sale[(sale[PRICE_COL] > 0) & (sale[AREA_COL] > 0)].copy()

# 월 단위 시계열 생성
sale["ym"] = sale[DATE_COL].dt.to_period("M").dt.to_timestamp()

# 단위면적 가격 (만원/㎡)
sale["unit_price"] = sale[PRICE_COL] / sale[AREA_COL]

In [7]:
# ===============================
# 2. 월별 가격지수 및 로그수익률 계산
# ===============================
# 월별 중앙값을 사용해 극단값 영향 완화
monthly = (
    sale.groupby("ym")["unit_price"]
    .agg(n="size", P="median")
    .sort_index()
)

# 거래 수가 너무 적은 월은 신뢰도가 낮으므로 제거
MIN_N = 10
monthly = monthly[monthly["n"] >= MIN_N].copy()

# 월 로그수익률 r_t = log(P_t / P_{t-1})
monthly["r_t"] = np.log(monthly["P"] / monthly["P"].shift(1))
monthly = monthly.dropna(subset=["r_t"]).copy()


# ===============================
# 3. μ, σ 추정 (연율화)
# ===============================
# GBM 가정 하에서:
# - μ : 평균 로그수익률
# - σ : 로그수익률의 표준편차
# 월 단위 추정 후 연율화
mu_hat = monthly["r_t"].mean() * 12
sigma_hat = monthly["r_t"].std(ddof=1) * np.sqrt(12)

print(f"[기존 추정] mu_hat (연율)    = {mu_hat:.6f}")
print(f"[기존 추정] sigma_hat (연율) = {sigma_hat:.6f}")


# ===============================
# 4. 금리 데이터 로드 및 전처리
# ===============================
rate = pd.read_csv(RATE_PATH)

# rate.csv는 '연월', '금리(%)' 구조라고 가정
rate["연월"] = rate["연월"].astype(str)
rate["_dt"] = pd.to_datetime(rate["연월"], format="%Y%m", errors="coerce")
rate["rate"] = pd.to_numeric(
    rate["금리(%)"].astype(str).str.replace("%", ""),
    errors="coerce"
)

rate = rate.dropna(subset=["_dt", "rate"]).copy()
rate["ym"] = rate["_dt"].dt.to_period("M").dt.to_timestamp()

# 월별 금리 (평균)
rate_m = (
    rate.groupby("ym")["rate"]
    .mean()
    .reset_index()
    .rename(columns={"rate": "rate_t"})
    .sort_values("ym")
)

# 월별 금리 변화 Δrate_t
rate_m["d_rate"] = rate_m["rate_t"].diff()
rate_m = rate_m.dropna(subset=["d_rate"]).copy()

[기존 추정] mu_hat (연율)    = -0.007366
[기존 추정] sigma_hat (연율) = 0.313363


In [8]:
# ===============================
# 5. 매매 수익률 ~ 금리 변화 회귀 (β 추정)
# ===============================
# r_t = α + β * Δrate_t + ε
df_reg = monthly.reset_index().merge(
    rate_m[["ym", "rate_t", "d_rate"]],
    on="ym",
    how="inner"
)

y = df_reg["r_t"].values
x = df_reg["d_rate"].values

# 단순 OLS (절편 포함)
X = np.column_stack([np.ones(len(x)), x])
coef = np.linalg.inv(X.T @ X) @ (X.T @ y)
alpha_reg, beta_raw = coef

print(f"[금리 민감도 추정] beta_raw = {beta_raw:.6f}")


# ===============================
# 6. β 안정화 (클리핑 + 수축)
# ===============================
# Track B의 목적은 '정확한 예측'이 아니라
# '현실적인 위험 반영'이므로 보수적으로 안정화

BETA_LOWER = -0.2   # 금리↑ → 가격↓ 관계만 허용
BETA_UPPER = 0.0
LAMBDA_SHRINK = 0.5

beta_clip = np.clip(beta_raw, BETA_LOWER, BETA_UPPER)
beta_final = LAMBDA_SHRINK * beta_clip

print("\n[beta 안정화]")
print(f"beta_raw   = {beta_raw:.6f}")
print(f"beta_clip  = {beta_clip:.6f}")
print(f"beta_final = {beta_final:.6f}")


# ===============================
# 7. μ 보정 (금리 반영)
# ===============================
# 기준 금리: 회귀에 사용된 기간 평균
r_hist_avg = df_reg["rate_t"].mean()

# 현재 금리: 가장 최근 월
r_current = rate_m.sort_values("ym")["rate_t"].iloc[-1]

# μ 보정
mu_adj = mu_hat + beta_final * (r_current - r_hist_avg)

# μ 폭주 방지 (연율 기준 ±10%p 제한)
MU_CLIP_DELTA = 0.10
mu_adj_clipped = np.clip(
    mu_adj,
    mu_hat - MU_CLIP_DELTA,
    mu_hat + MU_CLIP_DELTA
)

[금리 민감도 추정] beta_raw = -0.014638

[beta 안정화]
beta_raw   = -0.014638
beta_clip  = -0.014638
beta_final = -0.007319


In [9]:
print("\n[μ 보정 결과]")
print(f"mu_hist (연율)        = {mu_hat:.6f}")
print(f"mu_adj  (연율)        = {mu_adj:.6f}")
print(f"mu_adj_clipped (최종) = {mu_adj_clipped:.6f}")
print(f"Δmu                  = {mu_adj_clipped - mu_hat:.6f}")


[μ 보정 결과]
mu_hist (연율)        = -0.007366
mu_adj  (연율)        = -0.002734
mu_adj_clipped (최종) = -0.002734
Δmu                  = 0.004632
